# DLC Model Creation and Training Script

**RUNNING THIS SCRIPT:** Run this script with the DEEPLABCUT conda environment.

This script generates a DLC multi-animal model, which can even be used for single animals (in fact, training and inference is both faster too).

The script follows the below steps, which are almost identical to the prescribed process [here](https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html):

1. Create a DLC project folder and config file. The config file stores the model's data, training, and inference configurations.
1. Manually change the following parameters in the config file:
   <!-- * `identity: true` (as we can identify each animal uniquely across frames) -->
   - `individuals`: name for each animal (e.g. `mouse1`)
   - `uniquebodyparts`: parts in the arena that are NOT the animal (e.g. `TopLeft`, `ColourMarking`)
   - `multianimalbodyparts`: bodyparts for an animal (e.g. `Nose`)
   - `numframes2pick`: The number of frames to extract from each video for labeling. A rule of thumb is ~500 frames overall is sufficient to train a model well.
   <!-- * `batch_size: 32` (Speeds up computation for better GPUs). -->
1. Load videos to be used for training into the project's `videos` folder and update the config file with a list of these videos.
1. Randomly extract `n` (user specified) frames from each video and store in the `labeled-data` folder.
   - NOTE: it can be useful to trim videos and import these to the project to get frames that you'd particularly like to label (e.g. close interaction in social experiments).
1. Downsample all frames to `960 x 540 px` (or the resolution you'd like). Also update config file to reflect this resolution.
1. Manually label frames
1. Create combined training dataset from labeled frames
1. Run training. The following training parameters are usually ideal:
   - `saveiter = 5000`
   - `maxiter = 50000`
1. Evaluate statistic (optional - gives MAE but difficult to interrogate this single statistic).
1. Test on novel video(s) and manually inspect tracking.

NOTE: for experiments with multiple animals, the following parameters are usually ideal:

- TODO: in pose_inference.yaml


In [ ]:
import os
import re

import cv2
import shutil
import deeplabcut
import yaml
import numpy as np
import subprocess

## Specify project folder and name

DLC models are usually stored in the `Z:\PRJ-BowenLab\PRJ-BowenLab\DeepLabCut-Projects` folder.


In [ ]:
# CHANGE
root_dir = r"/home/linux1/Desktop/models_training/3_CHAMBER_CUPS"
# CHANGE
proj_name = "3_CHAMBER_CUPS_BLACK_MICE_960x540px"
# Don't need to change
experimenter = "BowenLab"
# Can modify if running multiple GPU's at once
gputouse = 0
# Downsampling size for training frames
# 960 x 540 is a good size
res_width = 960
res_height = 540

# DON'T CHANGE
proj_dir = os.path.join(root_dir, proj_name)
config_fp = os.path.join(proj_dir, "config.yaml")

display(os.path.isfile(config_fp))

## Creating project

NOTE: don't need to run if project is already created.


In [ ]:
# Only run if porject doesn't exist yet
if os.path.exists(proj_dir):
    print(f"NOT making project because it already exists: {proj_dir}")
else:
    # Making placeholder vid
    placeholder_fp = os.path.join(root_dir, "placeholder_vid.mp4")
    cap = cv2.VideoWriter(
        placeholder_fp,
        cv2.VideoWriter_fourcc(*"mp4v"),
        15,
        (res_width, res_height),
    )
    black_frame = np.zeros((res_height, res_width, 3), dtype=np.uint8)
    for _ in range(15):
        cap.write(black_frame)
    cap.release()
    # Creating project
    temp_config_fp = deeplabcut.create_new_project(
        project=proj_name,
        experimenter=experimenter,
        videos=[placeholder_fp],
        working_directory=root_dir,
        copy_videos=True,
        multianimal=True,
    )
    # Renaming project to just proj_name
    os.rename(src=os.path.dirname(temp_config_fp), dst=proj_dir)
    # Updating config file with:
    # animals are identifiable, updated project path, and batch size
    deeplabcut.auxiliaryfunctions.edit_config(
        config_fp,
        {
            "identity": True,
            "project_path": proj_dir,
            "batch_size": 8,
        },
    )
    # Remove placeholding videos from project videos folder
    os.remove(placeholder_fp)
    os.remove(os.path.join(proj_dir, "videos", "placeholder_vid.mp4"))
    # Making folder named "videos_raw" to store vids before downsampling
    os.makedirs(os.path.join(proj_dir, "videos_raw"), exist_ok=True)
    os.makedirs(os.path.join(proj_dir, "test_on_novels"), exist_ok=True)

## Manually change config file parameters

**ATTENTION**

Manually update the following parameters in the `config.yaml` file.

- `individuals`: name for each animal (e.g. `mouse1`)
- `uniquebodyparts`: parts in the arena that are NOT the animal (e.g. `TopLeft`, `ColourMarking`)
- `multianimalbodyparts`: bodyparts for an animal (e.g. `Nose`)
- `numframes2pick`: The number of frames to extract from each video for labeling. A rule of thumb is ~500 frames overall is sufficient to train a model well.


# Import raw training videos

**ATTENTION**

Copy training videos to the `<proj_dir>\videos_raw` folder.


## Downsampling videos and saving to the `videos` folder

Uses ffmpeg to downsample videos.

Set the `res_width` and `res_height` values in the 2nd top Python cell (that has all the other settings for training this model).


In [ ]:
def downsample_vid(in_fp, out_fp, res_width, res_height):
    cmd = [
        "ffmpeg",
        "-i",
        in_fp,
        "-vf",
        f"scale={res_width}:{res_height}",
        "-c:v",
        "h264",
        "-preset",
        "fast",
        "-crf",
        "20",
        "-y",
        out_fp,
    ]
    os.makedirs(os.path.dirname(out_fp), exist_ok=True)
    subprocess.run(cmd)


for vid_fp in os.listdir(os.path.join(proj_dir, "videos_raw")):
    downsample_vid(
        in_fp=os.path.join(proj_dir, "videos_raw", vid_fp),
        out_fp=os.path.join(proj_dir, "videos", vid_fp),
        res_width=res_width,
        res_height=res_height,
    )

### Updating configs file with the filepaths of our training videos

This is required for DLC's extract frames step.
It looks at the video filepaths in the configs file and extracts frames from those videos for labeling.


In [ ]:
def update_config_videos(proj_dir):
    # Getting folder names for videos and labeled data
    videos_dir = os.path.join(proj_dir, "videos")
    labeled_dir = os.path.join(proj_dir, "labeled-data")
    video_sets = {}
    # For each video, store vid dims in video_configs
    for j in os.listdir(videos_dir):
        vid_fp = os.path.join(videos_dir, j)
        # Getting video dimensions
        cap = cv2.VideoCapture(vid_fp)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        # Adding to video_sets dict
        video_sets[vid_fp] = {"crop": f"0, {width}, 0, {height}"}
        # Closing video
        cap.release()
    # For all labeled-data frames (corresponding to videos)
    # Overwrites the video dimensions because these are the actual frames
    # Used for training
    for i in os.listdir(labeled_dir):
        # Not considering labeled data
        if re.search("_labeled$", i):
            continue
        # Getting one of the image frames in the
        # labeled data dict (to get video dimensions)
        fp_ls = [j for j in os.listdir(os.path.join(labeled_dir, i)) if re.search("\.png$", j)]
        # If no frames, skip
        if len(fp_ls) == 0:
            continue
        # Getting frame fp and video fp
        vid_fp = os.path.join(videos_dir, f"{i}.mp4")
        png_fp = os.path.join(labeled_dir, i, fp_ls[0])
        # Getting video dimensions
        height, width, ch = cv2.imread(png_fp).shape
        video_sets[vid_fp] = {"crop": f"0, {width}, 0, {height}"}
    # Updating configs file with video_sets
    deeplabcut.auxiliaryfunctions.edit_config(config_fp, {"video_sets": video_sets})


update_config_videos(proj_dir)

# # Regular DLC implementation (does not consider extracted frame dimensions)
# deeplabcut.add_new_videos(
#     config_path,
#     r"z:\PRJ-BowenLab\PRJ-BowenLab\DeepLabCut-Projects\SFC_WHITE_MICE\videos",
#     copy_videos=True,
# )

## Extract frames

Randomly extract `n` (user specified) frames from each video and store in the `labeled-data` folder.

NOTE: edit the `numframes2pick` value in `configs.yaml` to change the number of frames extracted.
~500 frames overall is sufficient to train a model well.

NOTE: it can be useful to trim videos and import these to the project to get frames that you'd particularly like to label (e.g. close interaction in social experiments).


In [ ]:
# EXTRACTING FRAMES

# Getting folder names for videos and labeled data
videos_dir = os.path.join(proj_dir, "videos")
labeled_dir = os.path.join(proj_dir, "labeled-data")
# Get `n` from config file
with open(config_fp, "r") as f:
    config = yaml.safe_load(f)
n = config["numframes2pick"]
# For each video, extract `n` frames
for i in os.listdir(videos_dir):
    # Getting video fp
    vid_name = os.path.splitext(i)[0]
    vid_fp = os.path.join(videos_dir, i)
    vid_labeled_dir = os.path.join(labeled_dir, vid_name)
    # Opening video
    print(f"Extracting {n} frames from {vid_name}")
    vid = cv2.VideoCapture(vid_fp)
    # Getting total frames in video
    total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    # Getting `k` random frames
    frame_ids = np.random.choice(total_frames, n, replace=False).astype(int)
    # Creating folder for video
    os.makedirs(vid_labeled_dir, exist_ok=True)
    # For each frame in randomly selected set, save frame
    for j in sorted(frame_ids):
        print(f"    saving frame {j:06} ... ")
        # Seeking to frame
        vid.set(cv2.CAP_PROP_POS_FRAMES, j)
        # Reading frame
        ret, frame = vid.read()
        # Saving frame
        if ret:
            cv2.imwrite(os.path.join(vid_labeled_dir, f"img{j:06}.png"), frame)
    # Closing video
    vid.release()

# # Regular DLC implementation (any error will entirely halt the process)
# deeplabcut.extract_frames(
#     config_path,
#     mode="automatic",  # "automatic"/"manual"
#     algo='uniform',  # "uniform"/"kmeans"
#     userfeedback=False,  # True/False
#     crop=False  # keep as False
# )

## Label frames


In [ ]:
deeplabcut.label_frames(config_fp)

TODO: check that all frames are labelled without deleting rows and images


## Create training dataset


In [ ]:
# deeplabcut.create_training_dataset(config_fp)

deeplabcut.create_multianimaltraining_dataset(config_fp)

## Train model

Note the pytorch training configs. These particular configs are stored in the `dlc-models-pytorch/.../train/pytorch_config.yaml` file.


In [ ]:
deeplabcut.train_network(
    config_fp,
    shuffle=1,
    trainingsetindex=0,
    gputouse=gputouse,
    max_snapshots_to_keep=5,
    autotune=False,
    displayiters=100,
    # saveiters=5000,
    save_epochs=50,
    # maxiters=50000, # Can change - 50000 is good
    epochs=1000,
    allow_growth=True,
    pytorch_cfg_updates={
        "runner.gpus": [gputouse],
        "runner.snapshots.max_snapshots": 5,
        "runner.snapshots.save_epochs": 50,
        "runner.snapshots.save_optimizer_state": False,
        "train_settings.batch_size": 8,
        "train_settings.dataloader_workers": 1,
        "train_settings.dataloader_pin_memory": True,
        "train_settings.display_iters": 100,
        "train_settings.epochs": 1000,
        "train_settings.seed": 42,
    },
)

## Evaluate model

Optional - this gives a Mean Absolute Error, which is difficult to interrogate.

It is advisable to instead run the model on some novel videos and inspect its performance by eye.


In [ ]:
# deeplabcut.evaluate_network(config_fp, plotting=False)


## Test on novel video(s) and manually inspect tracking

Firstly, make a folder in `proj_dir` called `novel_videos` and add some novel videos.

Then run the following code block, which runs the model on these video.

Inspect these videos and if performance is not satisfactory, label more frames and rerun training.

Notes for inspection:

- Importantly, do bodypoints track well.
- For multi-animal experiments, do points assemble to a single animal well (even if the identity is incorrect),
- For multi-animal experiments, don't worry about swapping identities - a postprocessing step is done in our pipeline which fixes the identities to the markings/non-markings of each animal.


In [ ]:
novel_vids_dir = os.path.join(proj_dir, "test_on_novels")
assert os.path.exists(novel_vids_dir)

try:
    shutil.rmtree(os.path.join(novel_vids_dir, "out"))
except FileNotFoundError:
    pass
os.makedirs(os.path.join(novel_vids_dir, "out"), exist_ok=True)

deeplabcut.analyze_videos(
    config=config_fp,
    videos=os.path.join(novel_vids_dir, "in"),
    videotype=".mp4",
    destfolder=os.path.join(novel_vids_dir, "out"),
    auto_track=True,
    gputouse=gputouse,
    save_as_csv=False,
    calibrate=False,
    identity_only=False,
    allow_growth=True,
    # torch_kwargs={
    #     "device": [gputouse],
    # },
)

In [ ]:
deeplabcut.create_labeled_video(
    config=config_fp,
    videos=os.path.join(novel_vids_dir, "in"),
    videotype=".mp4",
    color_by="individual",
    destfolder=os.path.join(novel_vids_dir, "out"),
    overwrite=True,
)